In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.tree            import DecisionTreeRegressor
from sklearn.neural_network  import MLPRegressor
from sklearn.linear_model    import LinearRegression
from sklearn.ensemble        import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics         import mean_squared_error
from sklearn.metrics         import r2_score

In [33]:
train_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\Train.csv")

In [75]:
test_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\Test.csv")

In [35]:
train_df.isnull().sum()

id                                   0
name                                12
host_id                              0
host_name                           13
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       6982
reviews_per_month                 6982
calculated_host_listings_count       0
availability_365                     0
dtype: int64

In [36]:
def RemovalOfNulls(df):
    df = df.drop(['name','host_name'],axis=1)
    df['last_review'] = pd.to_datetime(df['last_review'])
    min_val = np.min(df['reviews_per_month'])
    min_row = df[df['reviews_per_month']==min_val].min()
    df['last_review'] = df['last_review'].fillna(min_row['last_review'])
    df['reviews_per_month'] = df['reviews_per_month'].fillna(min_row['reviews_per_month'])
    return df

In [37]:
train_df = RemovalOfNulls(train_df)

In [38]:
train_df.isnull().sum()

id                                0
host_id                           0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [ ]:
train_df['log_(price+1)'] = np.log1p(train_df['price'])


In [43]:
train_df = train_df.drop(['price'],axis=1)

In [40]:
train_df =train_df.drop(['id','host_id'],axis=1)

In [41]:
train_df =train_df.drop(['neighbourhood'],axis=1)

In [44]:
train_df.columns

Index(['neighbourhood_group', 'latitude', 'longitude', 'room_type',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'log_(price+1)'],
      dtype='object')

In [47]:
train_df_with_dummies = pd.get_dummies(train_df)

In [53]:
train_df_with_dummies = train_df_with_dummies.drop(['last_review'],axis=1)

In [54]:
train_df_with_dummies.columns

Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'log_(price+1)', 'neighbourhood_group_Bronx',
       'neighbourhood_group_Brooklyn', 'neighbourhood_group_Manhattan',
       'neighbourhood_group_Queens', 'neighbourhood_group_Staten Island',
       'room_type_Entire home/apt', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')

In [55]:
X=train_df_with_dummies.drop(['log_(price+1)'],axis=1)
y=train_df_with_dummies['log_(price+1)']

In [56]:
#y= target
#x = nums
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=24)

In [57]:
from sklearn.linear_model import LinearRegression

In [58]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred_train = lr.predict(X_train)
print('Train RMSE:',np.sqrt(mean_squared_error(y_train, lr.predict(X_train))))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test, lr.predict(X_test))))


Train RMSE: 0.4989193281313136
Test RMSE: 0.5043963781347076


## TEST DATA

In [59]:
test_id_col = test_df['id']

In [76]:
test_df.isnull().sum()

id                                   0
name                                 4
host_id                              0
host_name                            8
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                    0
last_review                       3070
reviews_per_month                 3070
calculated_host_listings_count       0
availability_365                     0
dtype: int64

In [77]:
test_df = RemovalOfNulls(test_df)

In [78]:
test_df.isnull().sum()

id                                0
host_id                           0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [79]:
test_df = test_df.drop(['last_review'],axis=1)

In [65]:
test_df =test_df.drop(['host_id','neighbourhood'],axis=1)

In [66]:
test_df_with_dummies = pd.get_dummies(test_df)

In [68]:
test_df_with_dummies['price'] = lr.predict(test_df_with_dummies)

In [69]:
test_df_with_dummies['price'] = np.exp(test_df_with_dummies['price'])-1

In [70]:
submissions_lr = pd.concat([test_id_col, test_df_with_dummies['price']], axis=1) 


In [71]:
submissions_lr.to_csv('submission_LinearReg_with_cat.csv', index=False) 

In [72]:
submissions_lr

,id,price
0,1525602,135.067298
1,30430185,123.951636
2,21354525,72.742494
3,35995074,78.619453
4,34392081,57.090775
...,...,...
14664,27730191,146.741172
14665,29676417,83.895169
14666,1813829,97.833246
14667,25017403,211.236491


In [73]:
submissions_lr.isnull().sum()

id       0
price    0
dtype: int64